# YOLOv5 Fine-tuning with COCO Dataset

This notebook demonstrates how to:
1. Download COCO dataset
2. Fine-tune YOLOv5 
3. Use the fine-tuned model for detection

In [1]:
# Install required packages
%pip install ultralytics
%pip install torch torchvision
%pip install pycocotools



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Clone YOLOv5 repository if not already done
!git clone https://github.com/ultralytics/yolov5
%cd yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 17067, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 17067 (delta 24), reused 28 (delta 12), pack-reused 17022 (from 1)
Receiving objects: 100% (17067/17067), 15.68 MiB | 1.04 MiB/s, done.
Resolving deltas: 100% (11714/11714), done.
/Users/alexs/PycharmProjects/MachineLearningForCV/TD3/yolov5


/Users/alexs/PycharmProjects/MachineLearningForCV/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
# Download COCO128 dataset (a smaller subset of COCO)
!bash data/scripts/get_coco128.sh

######################################################################## 100.0%#######################################      95.7%


In [ ]:
# After the imports and before training, add this code to filter the dataset
from pathlib import Path
import yaml
import shutil

# Create custom YAML
custom_yaml = """
path: ../datasets/coco128  # dataset root dir
train: images/train2017  # train images
val: images/train2017  # val images

# Classes
nc: 3  # number of classes
names: ['car', 'stop sign', 'traffic light']  # class names
"""

# Write custom YAML file
with open('data/custom_traffic.yaml', 'w') as f:
    f.write(custom_yaml)

# Filter and modify labels to only include our classes
def filter_labels(label_path, class_mapping):
    if not label_path.exists():
        return
    
    with open(label_path) as f:
        labels = f.readlines()
    
    filtered_labels = []
    for label in labels:
        values = label.strip().split()
        class_idx = int(values[0])
        if class_idx in class_mapping:
            # Replace old class index with new one
            values[0] = str(class_mapping[class_idx])
            filtered_labels.append(' '.join(values) + '\n')
    
    if filtered_labels:
        with open(label_path, 'w') as f:
            f.writelines(filtered_labels)
    else:
        # If no relevant objects, delete the label file and corresponding image
        img_path = label_path.parent.parent / 'images' / label_path.parent.name / (label_path.stem + '.jpg')
        if img_path.exists():
            img_path.unlink()
        label_path.unlink()

# Define class mapping (COCO class index -> our class index)
class_mapping = {
    2: 0,   # car
    11: 1,  # stop sign
    9: 2    # traffic light
}

# Process all label files
dataset_path = Path('datasets/coco128')
label_files = list((dataset_path / 'labels/train2017').glob('*.txt'))
for label_file in label_files:
    filter_labels(label_file, class_mapping)

## Fine-tune YOLOv5 on COCO128

In [4]:
!python train.py --data custom_traffic.yaml --weights yolov5s.pt --img 640 --epochs 100 --batch-size 16 --cache

train: weights=yolov5s.pt, cfg=, data=coco128.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-388-g882c35fc Python-3.11.10 torch-2.5.1 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, 

## Test Fine-tuned Model on Video

In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt --source ../test.mp4